# API SNCF: Les retards de train

__Auteur :__ 

Steve Caron

__Présentation :__ 

Ce script permet de requêter les informations sur les arrivées des trains en gare pour la journée d'hier. 

Il retourne les réponses des requêtes dans deux répertoires:

* data/arrivees/ : répertoire contenant les informations sur les arrivées en gare

* data/perturbations/ : répertoire contenant les informations sur les perturabations observées sur les arrivées en gare


__Inputs :__ 

Un fichier data/top{n}gare.json contenant les informations permettant de faire les requêtes pour chaques gares. Les enregistrements sont classés en fonction de la fréquentation des gares.

__Params :__

* CLEF_API : nom sous lequel est enregistrer la cle API

In [239]:
from dotenv import load_dotenv
import os
import requests
import json
import csv
import datetime
from dataclasses import dataclass,asdict

In [240]:
CLEF_API = "API_KEY"

In [241]:
def convertir_en_string(dt):
    '''Cette fonction convertit un datetime en chaîne de caractères'''
    if str is None:
        return None
    else:
        return datetime.datetime.strftime(dt,'%Y%m%dT%H%M%S')

In [242]:
def convertir_en_datetime(str):
    '''Cette fonction convertit une chaîne de caractères en datetime'''
    if str is None:
        return None
    else:
        return datetime.datetime.strptime(str,"%Y%m%dT%H%M%S")

In [243]:
def to_json(data,nom_fichier):
    '''Cette fonction permet d'enregistrer un fichier JSON'''
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [244]:
def to_csv(data,nom_fichier):
    '''Cette fonction permet d'enregistrer un fichier CSV'''
    with open(nom_fichier,"w", newline='', encoding='utf-8')as fc:
        writer = csv.DictWriter(fc,fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

In [245]:
def requete_api(code_gare,code_reseau,date):
    '''Cette fonction effectue une requête API pour collecter la liste des arrivées pour une gare spécifique sur un réseau spécifique '''

    base_url = "https://api.sncf.com/v1/coverage/sncf"
    #Requete sans le filtre sur les trains
    requete = f"{base_url}/stop_areas/{code_gare}/networks/{code_reseau}/arrivals?from_datetime={date}"
    reponse = requests.get(requete, auth=(api_key,""))
    reponse_json = reponse.json()
    to_json(reponse_json,"_test.json")
    
    return reponse_json

In [246]:
def requete_entre_dates(code_gare,code_reseau,date_min,date_max,liste_arrivee,liste_perturbation,compteur_requete):
    '''Cette fonction permet de faire des requêtes pour récupérer des données sur tous les enregistrements de la journée.
    Elle sépare en deux listes les informations concernant les départs et les informations concernant les arrivées'''

    derniere_requete = date_min

    print(f"Première requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau}")

    while derniere_requete < date_max:
        # Requete api
        reponse = requete_api(code_gare,code_reseau,derniere_requete)
        compteur_requete += 1
        # extrait les arrivées
        arrivees = reponse["arrivals"]
        # extrait les perturbations
        perturbations = reponse["disruptions"]
        # Ajoute chaque arrivées de la requete à la liste
        [liste_arrivee.append(arrivee) for arrivee in arrivees]
        # Ajoute chaque perturbations de la requete à la liste
        [liste_perturbation.append(perturbation)  for perturbation in perturbations]
        # Si il a moins de 10 arrivées, alors la prochaine requete ne donnera rien, on retourne donc directement les résultats
        if len(arrivees) < 10:
            print(f"Dernière requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau} // dernieres requetes {len(arrivees)} arrivées")
            return liste_arrivee,liste_perturbation,compteur_requete
        
        derniere_requete = arrivees[-1]["stop_date_time"]["base_arrival_date_time"]
    
    print(f"Dernière requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau}")
    
    return liste_arrivee,liste_perturbation,compteur_requete


In [247]:
def liste_id(nom_fichier):
    '''Cette fonction ouvre un fichier json et récupère une liste de toutes les clés d'un dictionnaire'''

    #Ouverture du fichier csv
    with open(nom_fichier,"r") as jsonfil:
        data_gare = json.load(jsonfil)
    toutes_id = data_gare["id"]
    liste_cles_ranked = []
    # J'ajoute toutes les clés du dictionnaire id dans une liste
    [liste_cles_ranked.append(cle) for cle in toutes_id.keys()]
    return data_gare,liste_cles_ranked

In [248]:
@dataclass
class Arrivee:
    gare_id:                            str | None
    departure_date_time:                datetime.datetime | None
    base_departure_date_time:           datetime.datetime | None
    arrival_date_time:                  datetime.datetime | None
    base_arrival_date_time:             datetime.datetime | None
    network:                            str | None
    ligne:                              str | None
    direction:                          str | None
    disruption_id:                      str | None

In [249]:
@dataclass
class Perturbation:
    perturbation_id:                        str | None
    debut:                                  datetime.datetime | None
    fin:                                    datetime.datetime | None
    effet:                                  str | None
    message:                                str | None

In [250]:
def extraire_donnees(data,liste_cles:list):
    '''Cette fonction permet d'extraire une donnée dans un dictionnaire'''
    try:
        if len(liste_cles) ==1:
            data = data[liste_cles[0]]
        #Gestion des cas ou la premiere clé est application_period ou message qui contient une liste, on récupère alors les données du premier element de la liste
        elif liste_cles[0] == "application_periods" or liste_cles[0] == "messages":
            data = data[liste_cles[0]][0][liste_cles[1]]
        #Gestion des cas ou la deuxieme clé est links qui contient une liste, on récupère alors les données du premier element de la liste
        elif liste_cles[1] == "links":
            data = data[liste_cles[0]][liste_cles[1]][0][liste_cles[2]]
        else:
            for une_cle in liste_cles:
                data = data[une_cle]
        return data
    except (KeyError,IndexError):
        return None

In [251]:
def collecte_donnees_arrivee(json_arrivee):
    '''Cette fonction instancie une arrivée et remplie les champs en extrayant les données d'un dictionnaire'''
    arrivee = Arrivee(
        gare_id = extraire_donnees(json_arrivee,["stop_point","stop_area","id"]),
        departure_date_time = convertir_en_datetime(extraire_donnees(json_arrivee,["stop_date_time","departure_date_time"])),
        base_departure_date_time = convertir_en_datetime(extraire_donnees(json_arrivee,["stop_date_time","base_departure_date_time"])),
        arrival_date_time = convertir_en_datetime(extraire_donnees(json_arrivee,["stop_date_time","arrival_date_time"])),
        base_arrival_date_time = convertir_en_datetime(extraire_donnees(json_arrivee,["stop_date_time","base_arrival_date_time"])),
        network = extraire_donnees(json_arrivee,["display_informations","network"]),
        ligne = extraire_donnees(json_arrivee,["display_informations","label"]),
        direction = extraire_donnees(json_arrivee,["display_informations","direction"]),
        disruption_id = extraire_donnees(json_arrivee,["display_informations","links","id"])
    )
    return asdict(arrivee)

In [252]:
def collecte_donnees_perturbation(json_perturbation):
    '''Cette fonction instancie une perturbation et remplie les champs en extrayant les données d'un dictionnaire'''
    perturbation = Perturbation(
        perturbation_id = extraire_donnees(json_perturbation,["id"]),
        debut = extraire_donnees(json_perturbation,["application_periods","begin"]),
        fin = extraire_donnees(json_perturbation,["application_periods","end"]),
        effet = extraire_donnees(json_perturbation,["severity","effect"]),
        message = extraire_donnees(json_perturbation,["messages","text"])
    )
    return asdict(perturbation)

In [253]:
def run(data_gare,cle,date_min,date_max):
    '''Cette fontion lance les fonctions pour traiter les données d'une gare'''
    code_gare = data_gare["id"].get(cle)
    nom_gare = data_gare["nom"].get(cle)
    print(f"Debut des requetes pour la gare: {nom_gare}")

    compteur_requete = 0
    liste_code_reseau = []
    liste_arrivees = []
    liste_perturbations = []

    #Récupération des identifiant réseaux pour la gare en cours
    [liste_code_reseau.append(reseau["id"]) for reseau in data_gare["networks"].get(cle)]

    # Je traite réseau par réseau
    for reseau in liste_code_reseau:
        liste_arrivees, liste_perturbations,compteur_requete = requete_entre_dates(code_gare,reseau,date_min,date_max,liste_arrivees,liste_perturbations,compteur_requete)
    
    liste_arrivees_clean=[]
    for arrivee in liste_arrivees:
        data_clean_arrivees = collecte_donnees_arrivee(arrivee)
        liste_arrivees_clean.append(data_clean_arrivees)
    
    liste_perturbations_clean=[]
    for perturbation in liste_perturbations:
        data_clean_perturbation =collecte_donnees_perturbation(perturbation)
        liste_perturbations_clean.append(data_clean_perturbation)
        
    
    print(f"Fin des requetes pour la gare :{nom_gare} \n {compteur_requete} requetes effectuées")
    
    nom_fichier_arrivees = f"data/arrivees/{code_gare}-{date_max}.json".replace(":","_")
    nom_ficher_arrivees_clean = f"data/arrivees_propres/{code_gare}-{date_max}.csv".replace(":","_")
    nom_fichier_perturbations = f"data/perturbations/{code_gare}-{date_max}.json".replace(":","_")
    nom_ficher_perturbations_clean = f"data/perturbations_propres/{code_gare}-{date_max}.csv".replace(":","_")
    
    to_json(liste_arrivees,nom_fichier_arrivees)
    to_csv(liste_arrivees_clean,nom_ficher_arrivees_clean)
    to_json(liste_perturbations,nom_fichier_perturbations)
    to_csv(liste_perturbations_clean,nom_ficher_perturbations_clean)

In [254]:
# Récupération de la clé API
load_dotenv()
api_key = os.getenv(CLEF_API)

In [255]:
# Creation des string date permettant de faire les requete API 
aujourdhui = datetime.date.today()
hier_debut_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=0, minute=0 ,second=0)
hier_fin_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=23, minute=59 ,second=59)
date_min = convertir_en_string(hier_debut_journee)
date_max = convertir_en_string(hier_fin_journee)

In [ ]:
# Récupération des clés dictionnaire contenant les informations des gares
data_gare,liste_cle = liste_id("data/top200gare.json")
# Exécution du run pour chaque gare
for cle in liste_cle[:1]:
    run(data_gare,cle,date_min,date_max)